## Migrating Oozie Workflows to Airflow CDE DAGs

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install pandas
!pip install requests_toolbelt
!pip install xmltodict

In [ ]:
import migration_utility.workflow as workflow
import migration_utility.cdejob as cdejob
import migration_utility.cderesource as cderesource
import os

In [ ]:
project_input_dir = 'input'
project_output_dir = 'output'

dag_file = 'combined.py'
dag_name = 'combined'

cde_prefix = 'jmontenaro_combined'
cde_resource_name = 'resource'
cde_job_name = 'job'

hive_connection = 'default-hive-aws'

In [ ]:
os.environ["WORKLOAD_USER"] = ""
os.environ["WORKLOAD_PASSWORD"] = ""
os.environ["JOBS_API_URL"] = ""

In [ ]:
!mkdir -p {project_output_dir}

In [ ]:
ow = workflow.OozieWorkflow(project_input_dir)

In [ ]:
workflow_dict = ow.create_workflow_dict()
workflow_dict

In [ ]:
workflow_props = ow.create_workflow_props()
workflow_props

In [ ]:
workflow_dict = ow.replace_workflow_props(workflow_dict, workflow_props)
workflow_dict

In [ ]:
cj = cdejob.CDEJob(workflow_dict, hive_connection, cde_prefix, cde_resource_name, dag_name)

In [ ]:
cj.initialize_dag(project_output_dir, dag_file)
cj.dag_imports(project_output_dir, dag_file)
cj.dag_declaration('jmontenaro', project_output_dir, dag_file)

In [ ]:
spark_payloads = cj.parse_oozie_workflow(project_output_dir, dag_file, workflow_dict)
spark_payloads

In [ ]:
with open(project_output_dir + "/" + dag_file, 'r') as f:
    print(f.read())

In [ ]:
airflow_cde_payload = cj.oozie_to_cde_airflow_payload(dag_file, cde_resource_name, cde_job_name)
airflow_cde_payload

In [ ]:
cr = cderesource.CDEResource(os.environ["JOBS_API_URL"], os.environ["WORKLOAD_USER"], cde_prefix, cde_resource_name)

In [ ]:
token = cr.set_cde_token(os.environ["WORKLOAD_PASSWORD"])

In [ ]:
cr.create_cde_resource(token, cde_resource_name)

In [ ]:
cr.upload_file(cde_resource_name, project_input_dir, "pi.scala", token)

In [ ]:
for spark_payload in spark_payloads:
    cr.create_job_from_resource(token, spark_payload)

In [ ]:
cr.upload_file(cde_resource_name, project_output_dir, dag_file, token)

In [ ]:
cr.create_job_from_resource(token, airflow_cde_payload)